In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the page to scrape
url = "https://fbref.com/en/matches/46d9048f/Huddersfield-Town-Burnley-July-29-2022-Championship"

# Get the game ID from the URL
game_id = url.split("/")[-2]

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find the Shots table on the page
shots_table = soup.find("div", {"id": "switcher_shots"})

# Create empty lists to store data and create the columns
data = []
headers = ['Game_ID', 'Minute', 'Player', 'Team', 'xG', 'PSxG', 'Result', 'Distance', 'Body Part', 'Notes', 'SCA 1 Player', 'Event', 'SCA 2 Player', 'Event']

# Extract table rows
rows = shots_table.find("tbody").find_all("tr")
for row in rows:
    row_data = [game_id] + [cell.get_text().strip() for cell in row.find_all(["th", "td"])]
    data.append(row_data)

# Create a DataFrame from the scraped data
shots_df = pd.DataFrame(data, columns=headers)


In [26]:
both_teams = soup.find_all("th", {"colspan": "2"})

In [27]:
both_teams_2 = soup.find_all("div", {"class": "lineup"})

In [28]:
# Your existing code...

# Find the home and away teams and formations
team_headers = soup.find_all("th", {"colspan": "2"})
teams = [header.text.strip().split(" (")[0] for header in team_headers if "(" in header.text.strip()]
formations = [header.text.strip().split(" (")[1][:-1] for header in team_headers if "(" in header.text.strip()]

# Extracting the home and away teams and formations
home_team = teams[0]
away_team = teams[1]
home_formation = formations[0]
away_formation = formations[1]

# Your existing code...

# Print the home and away teams and formations
print(f"Home Team: {home_team}")
print(f"Away Team: {away_team}")
print(f"Home Formation: {home_formation}")
print(f"Away Formation: {away_formation}")


Home Team: Huddersfield
Away Team: Burnley
Home Formation: 4-1-4-1
Away Formation: 4-3-3


In [39]:
# Your existing code...

# Create a DataFrame from the scraped data
shots_df = pd.DataFrame(data, columns=headers)

# Add the team, away team, formation, and opponent formation information to the DataFrame
shots_df['Home_Team'] = home_team
shots_df['Away Team'] = away_team
shots_df['Formation'] = home_formation
shots_df['Opponent Formation'] = away_formation

# Print the DataFrame


     Game_ID Minute                 Player          Team    xG  PSxG  \
0   46d9048f     11            Josh Cullen       Burnley  0.07  0.02   
1   46d9048f     14          Ashley Barnes       Burnley  0.06  0.34   
2   46d9048f     18            Ian Maatsen       Burnley  0.05  0.28   
3   46d9048f     20         Connor Roberts       Burnley  0.21  0.88   
4   46d9048f     24         Josh Brownhill       Burnley  0.03  0.20   
5   46d9048f     38            Josh Koroma  Huddersfield  0.04         
6   46d9048f     40         Dara Costelloe       Burnley  0.23         
7   46d9048f     44         Samuel Bastien       Burnley  0.06         
8   46d9048f     45         Dara Costelloe       Burnley  0.06         
9   46d9048f                                                           
10  46d9048f     69         Josh Brownhill       Burnley  0.03         
11  46d9048f     74             Danny Ward  Huddersfield  0.05         
12  46d9048f     76  Taylor Harwood-Bellis       Burnley  0.04  

In [ ]:
#Add logic for is_home_team_shot and is_away_team_shot